In [1]:
import ipywidgets as widgets
from traitlets import Unicode

from api import VspModel

In [2]:
filename = 'test.vsp3'

In [3]:
model = VspModel(filename=filename)
model.load_design('test.des')

In [4]:
model.wetted_areas

{'Fuselage0': 289.16385788321963,
 'Horizontal0': 49.5789109263189,
 'Vertical0': 29.412904431874512,
 'Wing0': 108.39279963009967,
 'total': 476.5484728715127}

In [5]:
model.Fuselage.Design.Length = 15

In [6]:
model.design.WingLocation = 0.25

In [7]:
%%html

<style>
    .threejsVisualizer{
        font-family: Monospace;
        background-color: #000;
        color: #fff;
        margin: 0px;
        overflow: hidden;
    }
    .info {
        color: #fff;
        position: absolute;
        top: 10px;
        width: 100%;
        text-align: center;
        z-index: 100;
        display:block;
    }
</style>

In [8]:
%%javascript

require.config({
   "paths":{
      "Detector": "https://cdn.rawgit.com/mrdoob/three.js/master/examples/js/Detector",
      "TrackballControls": "https://rawgit.com/mrdoob/three.js/34fff87eacc4ed2034a01ac276484ab85060f75a/examples/js/controls/TrackballControls",
      "STLLoader": "https://cdn.rawgit.com/mrdoob/three.js/master/examples/js/loaders/STLLoader",
      "stats.min": "https://cdn.rawgit.com/mrdoob/three.js/master/examples/js/libs/stats.min",
      "three.min": "https://cdn.rawgit.com/mrdoob/three.js/master/build/three.min"
   },
   "shim": {
      "Detector": {
         "deps": ["three.min"]
      },
      "TrackballControls": {
         "deps": ["three.min"]
      },
      "STLLoader": {
         "deps": ["three.min"]
      },
      "stats.min": {
         "deps": ["three.min"],
          "exports": "Stats"
      },
      "three.min": {
         "deps": [],
         "exports":"THREE"
      }
   }
});

require(
[
    "nbextensions/widgets/widgets/js/widget",
    "nbextensions/widgets/widgets/js/manager",
    'stats.min', 'three.min', 'TrackballControls', 'STLLoader', 'Detector'
],

function(widget, manager, Stats, THREE){
    console.log("threejsView was required");
    var scope = {}
    var threejsView = widget.DOMWidgetView.extend({
        render: function(){
            var self = this;
            scope = self
            
            if ( ! Detector.webgl ) Detector.addGetWebGLMessage();

            var container, stats;

            var camera, controls, scene, renderer;

            var cross;

            init();
            animate();

            function init() {
                self.camera = new THREE.PerspectiveCamera( 60, 4/3, 0.01, 1e10 );
                self.camera.position.z = 0.2;
                
                // renderer
                self.renderer = new THREE.WebGLRenderer( { antialias: false } );
                self.renderer.setPixelRatio( window.devicePixelRatio );
                self.renderer.setSize( element.innerWidth(), 3/4 * element.innerWidth() );

                container = self.el; //document.createElement( 'div' );
                document.body.appendChild( container );
                container.appendChild( self.renderer.domElement );
                
                self.controls = new THREE.TrackballControls( self.camera , container );
                
                self.controls.rotateSpeed = 5.0;
                self.controls.zoomSpeed = 5;
                self.controls.panSpeed = 2;

                self.controls.noZoom = false;
                self.controls.noPan = false;

                self.controls.staticMoving = true;
                self.controls.dynamicDampingFactor = 0.3;

                self.scene = new THREE.Scene();

                self.scene.add( self.camera );

                // light
                var dirLight = new THREE.DirectionalLight( 0xffffff );
                dirLight.position.set( 200, 200, 1000 ).normalize();

                self.camera.add( dirLight );
                self.camera.add( dirLight.target );

                self.material = new THREE.MeshLambertMaterial( { color:0xffffff, side: THREE.DoubleSide } );

                self.loader = new THREE.STLLoader();
                
                self.value = JSON.parse(self.model.get('value'))
                
                for(var i in self.value){
                    self.value[i].geometry = self.loader.parse(self.value[i].polyData)
                    self.value[i].mesh = new THREE.Mesh(self.value[i].geometry, self.material)
                    self.value[i].mesh.position.setX(self.value[i].x)
                    self.value[i].mesh.position.setY(self.value[i].y)
                    self.value[i].mesh.position.setZ(self.value[i].z)
                    self.scene.add(self.value[i].mesh)
                }

                stats = new Stats();
                stats.domElement.style.position = 'absolute';
                stats.domElement.style.top = '0px';
                stats.domElement.style['z-index'] = 1;
                container.style.position = 'relative';
                container.appendChild( stats.domElement );

                //

                window.addEventListener( 'resize', onWindowResize, false );

            }

            function onWindowResize() {

                self.camera.aspect = 4/3;
                self.camera.updateProjectionMatrix();

                self.renderer.setSize( element.innerWidth(), 3/4 * element.innerWidth() );

                self.controls.handleResize();

            }

            function animate() {

                requestAnimationFrame( animate );

                self.controls.update();
                self.renderer.render( self.scene, self.camera );

                stats.update();

            }
            
            this.model.on('change:value', this.update, this);
        },
        
        update: function(){
            scope
            self.value = JSON.parse(self.model.get('value'))
        }
    });
    
    // Register the widget with the widget manager.
    manager.WidgetManager.register_widget_view('threejsView', threejsView);
});

<IPython.core.display.Javascript object>

In [9]:
class threejsVisualizer(widgets.DOMWidget):
    _view_name = Unicode('threejsView').tag(sync=True)
    value = Unicode('{\"name\":\"Missing Value Property\"}').tag(sync=True)

In [15]:
model.design.Length = 12

In [16]:
g.value = model.threejs_data

In [17]:
g = threejsVisualizer(value=model.threejs_data)

In [18]:
g

# Find methods in OpenVSP SWIG interface

In [11]:
import vsp

[d for d in dir(vsp) if 'parm' in d.lower()]

['FindContainerParmIDs',
 'FindParm',
 'GetBoolParmVal',
 'GetGeomParmIDs',
 'GetIntParmVal',
 'GetParm',
 'GetParmContainer',
 'GetParmDisplayGroupName',
 'GetParmGroupName',
 'GetParmLowerLimit',
 'GetParmName',
 'GetParmType',
 'GetParmUpperLimit',
 'GetParmVal',
 'GetXSecParm',
 'GetXSecParmIDs',
 'ResetXSecSkinParms',
 'SetParmDescript',
 'SetParmLowerLimit',
 'SetParmUpperLimit',
 'SetParmVal',
 'SetParmValLimits',
 'SetParmValUpdate',
 'VSP_CANT_FIND_PARM',
 'VSP_CANT_SET_NOT_EQ_PARM',
 'ValidParm']